In [0]:
-- INSIGHT 1: Relação HDI x Taxa de Suicídio (Paradoxo Nórdico?)
-- Pergunta: Países com HDI alto têm taxas maiores de suicídio?
-- ============================================================================
SELECT 
    hdi_category AS categoria_idh,
    ROUND(AVG(suicide_rate), 2) AS taxa_media_suicidio,
    ROUND(STDDEV(suicide_rate), 2) AS desvio_padrao,
    COUNT(DISTINCT country_code) AS numero_paises,
    ROUND(MIN(suicide_rate), 2) AS taxa_minima,
    ROUND(MAX(suicide_rate), 2) AS taxa_maxima
FROM gold.data_suicide.suicide_hdi_analysis
WHERE year >= 2015  -- Últimos anos
GROUP BY hdi_category
ORDER BY taxa_media_suicidio DESC;

In [0]:
WITH ultimo_ano AS (
    SELECT MAX(year) AS ano_maximo
    FROM gold.data_suicide.suicide_hdi_analysis
),
mudancas_recentes AS (
    SELECT 
        country_name AS pais,
        region AS regiao,
        hdi_category AS categoria_idh,
        year AS ano,
        suicide_rate AS taxa_suicidio,
        LAG(suicide_rate, 5) OVER (PARTITION BY country_code ORDER BY year) AS taxa_5_anos_atras,
        try_divide(
            suicide_rate - LAG(suicide_rate, 5) OVER (PARTITION BY country_code ORDER BY year),
            LAG(suicide_rate, 5) OVER (PARTITION BY country_code ORDER BY year)
        ) * 100 AS mudanca_percentual_5anos
    FROM gold.data_suicide.suicide_hdi_analysis
)
SELECT 
    pais,
    regiao,
    categoria_idh,
    ROUND(taxa_5_anos_atras, 2) AS taxa_ha_5_anos,
    ROUND(taxa_suicidio, 2) AS taxa_atual,
    ROUND(mudanca_percentual_5anos, 2) AS mudanca_percentual
FROM mudancas_recentes
CROSS JOIN ultimo_ano
WHERE ano = ano_maximo
  AND taxa_5_anos_atras IS NOT NULL
ORDER BY mudanca_percentual_5anos DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
-- INSIGHT 3: Disparidade de Gênero - Países com maior diferença
-- ============================================================================
SELECT 
    country_name AS pais,
    year AS ano,
    ROUND(male_rate, 2) AS taxa_masculina,
    ROUND(female_rate, 2) AS taxa_feminina,
    ROUND(gender_ratio, 2) AS razao_masculino_feminino,
    ROUND(gender_gap, 2) AS diferenca_absoluta,
    gap_category AS categoria_diferenca,
    ROUND(hdi, 3) AS idh
FROM gold.data_suicide.gender_disparity_analysis
WHERE year = (SELECT MAX(year) FROM gold.data_suicide.gender_disparity_analysis)
ORDER BY razao_masculino_feminino DESC
LIMIT 20;

In [0]:
-- INSIGHT 4: Evolução das taxas por região ao longo do tempo
-- ============================================================================
SELECT 
    region AS regiao,
    year AS ano,
    ROUND(AVG(suicide_rate), 2) AS taxa_media_regional,
    COUNT(DISTINCT country_code) AS paises_reportando,
    ROUND(MIN(suicide_rate), 2) AS minima_na_regiao,
    ROUND(MAX(suicide_rate), 2) AS maxima_na_regiao
FROM gold.data_suicide.suicide_hdi_analysis
GROUP BY region, year
ORDER BY region, year;

In [0]:
-- INSIGHT 5: Países que conseguiram reduzir significativamente (Casos de Sucesso)
-- ============================================================================
WITH analise_tendencia AS (
    SELECT 
        country_code AS codigo_pais,
        country_name AS pais,
        region AS regiao,
        MIN(year) AS primeiro_ano,
        MAX(year) AS ultimo_ano,
        FIRST_VALUE(suicide_rate) OVER (
            PARTITION BY country_code 
            ORDER BY year ASC 
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS taxa_inicial,
        LAST_VALUE(suicide_rate) OVER (
            PARTITION BY country_code 
            ORDER BY year ASC 
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS taxa_final,
        ROUND(AVG(hdi), 3) AS idh_medio
    FROM gold.data_suicide.suicide_hdi_analysis
    WHERE year >= 2000
    GROUP BY country_code, country_name, region, suicide_rate, year, hdi
)
SELECT DISTINCT
    pais,
    regiao,
    primeiro_ano,
    ultimo_ano,
    ROUND(taxa_inicial, 2) AS taxa_no_inicio,
    ROUND(taxa_final, 2) AS taxa_no_fim,
    ROUND(((taxa_final - taxa_inicial) / taxa_inicial * 100), 2) AS reducao_percentual,
    idh_medio
FROM analise_tendencia
WHERE taxa_inicial > taxa_final
  AND ((taxa_inicial - taxa_final) / taxa_inicial) > 0.3  -- Redução de +30%
  AND (ultimo_ano - primeiro_ano) >= 10  -- Pelo menos 10 anos de dados
ORDER BY reducao_percentual ASC
LIMIT 15;

In [0]:
-- INSIGHT 6: Alertas críticos - Países que precisam de atenção
-- ============================================================================
SELECT 
    country_name AS pais,
    year AS ano,
    ROUND(suicide_rate, 2) AS taxa_atual,
    ROUND(rate_change_pct, 2) AS mudanca_1_ano_pct,
    ROUND(change_5y_pct, 2) AS mudanca_5_anos_pct,
    alert_type AS tipo_alerta,
    ROUND(hdi, 3) AS idh
FROM gold.data_suicide.significant_changes_alerts
WHERE year >= 2015
ORDER BY ABS(mudanca_1_ano_pct) DESC;

In [0]:
-- INSIGHT 7: Performance por década - Evolução global
-- ============================================================================
SELECT 
    decade AS decada,
    ROUND(AVG(suicide_rate), 2) AS taxa_media,
    COUNT(DISTINCT country_code) AS numero_paises,
    ROUND(AVG(hdi), 3) AS idh_global_medio,
    ROUND(STDDEV(suicide_rate), 2) AS desvio_padrao
FROM gold.data_suicide.suicide_hdi_analysis
GROUP BY decade
ORDER BY decade;

In [0]:
-- INSIGHT 8: Correlação entre taxa de suicídio e HDI (análise estatística)
-- ============================================================================
SELECT 
    hdi_category AS categoria_idh,
    year AS ano,
    ROUND(AVG(suicide_rate), 2) AS taxa_media,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY suicide_rate), 2) AS mediana,
    ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY suicide_rate), 2) AS percentil_25,
    ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY suicide_rate), 2) AS percentil_75,
    COUNT(*) AS observacoes
FROM gold.data_suicide.suicide_hdi_analysis
WHERE year >= 2010
GROUP BY hdi_category, year
ORDER BY year DESC, hdi_category;

In [0]:
-- INSIGHT 9: Rankings - Piores e Melhores países (últimos dados disponíveis)
-- ============================================================================
WITH ranking_recente AS (
    SELECT *
    FROM gold.data_suicide.country_rankings
    WHERE year = (SELECT MAX(year) FROM gold.data_suicide.country_rankings)
)
(
    -- TOP 10 PIORES
    SELECT 
        'Maiores Taxas' AS categoria,
        country_name AS pais,
        region AS regiao,
        ROUND(suicide_rate, 2) AS taxa,
        rank_global AS ranking_global,
        rank_in_region AS ranking_regional
    FROM ranking_recente
    ORDER BY suicide_rate DESC
    LIMIT 10
)
UNION ALL
(
    -- TOP 10 MELHORES
    SELECT 
        'Menores Taxas' AS categoria,
        country_name AS pais,
        region AS regiao,
        ROUND(suicide_rate, 2) AS taxa,
        rank_global AS ranking_global,
        rank_in_region AS ranking_regional
    FROM ranking_recente
    ORDER BY suicide_rate ASC
    LIMIT 10
)
ORDER BY categoria DESC, taxa DESC;

In [0]:
-- INSIGHT 10: Países acima/abaixo da média regional
-- ============================================================================
SELECT 
    country_name AS pais,
    region AS regiao,
    ROUND(suicide_rate, 2) AS taxa_pais,
    ROUND(regional_avg_rate, 2) AS media_regional,
    ROUND(vs_regional_avg, 2) AS diferenca_da_media,
    CASE 
        WHEN vs_regional_avg > 5 THEN 'Muito Acima da Média'
        WHEN vs_regional_avg > 0 THEN 'Acima da Média'
        WHEN vs_regional_avg > -5 THEN 'Abaixo da Média'
        ELSE 'Muito Abaixo da Média'
    END AS categoria_performance
FROM gold.data_suicide.country_rankings
WHERE year = (SELECT MAX(year) FROM gold.data_suicide.country_rankings)
  AND ABS(vs_regional_avg) > 3  -- Diferença significativa
ORDER BY ABS(diferenca_da_media) DESC
LIMIT 30;

In [0]:
-- INSIGHT BRASIL 1: Evolução histórica do Brasil
-- ============================================================================
SELECT 
    year AS ano,
    ROUND(suicide_rate, 2) AS taxa_brasil,
    ROUND(regional_avg_rate, 2) AS media_americas,
    ROUND(vs_regional_avg, 2) AS brasil_vs_media,
    rank_in_region AS ranking_regional,
    rank_global AS ranking_global
FROM gold.data_suicide.country_rankings
WHERE country_code = 'BRA'
ORDER BY year;

Databricks visualization. Run in Databricks to view.

In [0]:
-- INSIGHT BRASIL 2: Disparidade de gênero no Brasil
-- ============================================================================
SELECT 
    year AS ano,
    ROUND(male_rate, 2) AS taxa_masculina,
    ROUND(female_rate, 2) AS taxa_feminina,
    ROUND(gender_ratio, 2) AS razao_m_f,
    ROUND(gender_gap, 2) AS diferenca_absoluta,
    gap_category AS categoria_diferenca,
    ROUND(hdi, 3) AS idh_brasil
FROM gold.data_suicide.gender_disparity_analysis
WHERE country_code = 'BRA'
ORDER BY year;

In [0]:
-- INSIGHT BRASIL 3: Brasil comparado com países BRICS
-- ============================================================================
SELECT 
    country_name AS pais,
    year AS ano,
    ROUND(suicide_rate, 2) AS taxa_suicidio,
    ROUND(hdi, 3) AS idh,
    hdi_category AS categoria_idh,
    trend AS tendencia
FROM gold.data_suicide.suicide_hdi_analysis
WHERE country_code IN ('BRA', 'CHN', 'IND', 'RUS', 'ZAF')  -- BRICS
  AND year >= 2010
ORDER BY year DESC, taxa_suicidio DESC;

In [0]:
-- INSIGHT BRASIL 4: Brasil comparado com países da América Latina
-- ============================================================================
WITH america_latina AS (
    SELECT country_code
    FROM gold.data_suicide.suicide_hdi_analysis
    WHERE region LIKE '%America%'
      AND country_code IN ('BRA', 'ARG', 'CHL', 'COL', 'MEX', 'PER', 'URY')
    GROUP BY country_code
),
dados_latam AS (
    SELECT 
        s.country_name AS pais,
        s.year AS ano,
        ROUND(s.suicide_rate, 2) AS taxa_suicidio,
        ROUND(s.hdi, 3) AS idh,
        s.trend AS tendencia,
        RANK() OVER (
            PARTITION BY s.year
            ORDER BY s.suicide_rate DESC
        ) AS ranking_regional
    FROM gold.data_suicide.suicide_hdi_analysis s
    INNER JOIN america_latina al ON s.country_code = al.country_code
    WHERE s.year >= 2015
)
SELECT *
FROM dados_latam
ORDER BY ano DESC, taxa_suicidio DESC;

Databricks visualization. Run in Databricks to view.

In [0]:
-- INSIGHT FUTURO 1: Correlação entre tratamento de drogas e taxa de suicídio
-- (Descomentar quando silver.data_suicide.drug_indicators_wide estiver pronta)
-- ============================================================================

SELECT 
    s.country_name AS pais,
    s.year AS ano,
    ROUND(s.suicide_rate, 2) AS taxa_suicidio,
    ROUND(d.treatment_coverage, 2) AS cobertura_tratamento,
    ROUND(d.capacity_index, 2) AS indice_capacidade,
    ROUND(s.hdi, 3) AS idh,
    s.hdi_category AS categoria_idh
FROM gold.data_suicide.suicide_hdi_analysis s
INNER JOIN silver.data_suicide.drug_indicators_wide d
    ON s.country_code = d.country_code
    AND s.year = d.year
WHERE d.treatment_coverage IS NOT NULL
  AND s.year >= 2015
ORDER BY s.year DESC, s.suicide_rate DESC;

In [0]:
-- INSIGHT FUTURO 2: Países com alta capacidade de tratamento vs. taxa de suicídio
-- ============================================================================

SELECT 
    s.country_name AS pais,
    ROUND(AVG(s.suicide_rate), 2) AS taxa_media_suicidio,
    ROUND(AVG(d.capacity_index), 2) AS indice_capacidade_medio,
    ROUND(AVG(d.treatment_coverage), 2) AS cobertura_media_tratamento,
    ROUND(AVG(s.hdi), 3) AS idh_medio,
    COUNT(*) AS anos_com_dados
FROM gold.data_suicide.suicide_hdi_analysis s
INNER JOIN silver.data_suicide.drug_indicators_wide d
    ON s.country_code = d.country_code
    AND s.year = d.year
WHERE s.year >= 2010
GROUP BY s.country_name
HAVING COUNT(*) >= 5  -- Pelo menos 5 anos de dados
ORDER BY taxa_media_suicidio DESC;
